In [1]:
import os
import subprocess
import shutil
from glob import glob
import nibabel as nib
import numpy as np
import monai
import torch
monai.config.print_config()

MONAI version: 1.3.0
Numpy version: 1.21.6
Pytorch version: 1.13.0+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /home/<username>/.local/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.21.0
scipy version: 1.8.0
Pillow version: 9.5.0
Tensorboard version: 2.8.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.2
einops version: 0.4.1
transformers version: 4.24.0
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: 1.0.0
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:

# Only use MONAI / NIBABEL / ANTS

In [2]:
datasets = ['Feminad', 'Painfact', 'IRIS', 'GIN']
datasets = ['GIN']

# N4 with ANTS

In [19]:
#def n4(mri, out):
#    subprocess.run(["N4BiasFieldCorrection", "-d", "3", "-i", mri, "-o", out])

In [20]:
#for dataset in datasets:
#    print(dataset, end='\r')
#    mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*.nii.gz")))
#    for mri in mris:
#        if not os.path.exists(mri.replace(".nii", "_N4.nii")):
#            out = mri.replace(".nii", "_N4.nii")
#            n4(mri, out)

In [21]:
#for dataset in datasets:
#    print(dataset, end='\r')
#    masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*.nii.gz")))
#    for mask in masks:
#        if not os.path.exists(mask.replace(".nii", "_N4.nii")):
#            out = mask.replace(".nii", "_N4.nii")
#            shutil.copyfile(mask, out)
#    if dataset == 'GIN':   
#        labels = sorted(glob(os.path.join('datasets', dataset, 'Labels', "*.nii.gz")))
#        for label in labels:
#            if not os.path.exists(label.replace(".nii", "_N4.nii")):
#                out = label.replace(".nii", "_N4.nii")
#                shutil.copyfile(label, out)

# Change Affine for GIN

In [5]:
for dataset in datasets:
    if dataset == 'GIN':
        print(dataset, end='\r')
        mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*_N4.nii.gz")))
        masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*_N4.nii.gz")))
        labels = sorted(glob(os.path.join('datasets', dataset, 'Labels', "*_N4.nii.gz")))
        loader = monai.transforms.LoadImage(ensure_channel_first=True)    
        saver_mri = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/MRI/', output_postfix='o', separate_folder=False)  
        saver_mask = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Mask/', output_postfix='o', separate_folder=False) 
        saver_label = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Labels/', output_postfix='o', separate_folder=False)  
        for mri in mris:
            tensor = loader(mri)
            tensor.affine[:3,3] = torch.DoubleTensor([-7.6800,-7.6800,-7.6800])
            saver_mri(tensor) 
        for mask in masks:
            tensor = loader(mask)
            tensor.affine[:3,3] = torch.DoubleTensor([-7.6800,-7.6800,-7.6800])
            saver_mask(tensor)                
        for label in labels:
            tensor = loader(label)
            tensor.affine[:3,3] = torch.DoubleTensor([-7.6800,-7.6800,-7.6800])
            saver_label(tensor)          

2023-11-14 10:52:03,645 INFO image_writer.py:197 - writing: datasets/GIN/MRI/653_N4_o.nii.gz
2023-11-14 10:52:06,876 INFO image_writer.py:197 - writing: datasets/GIN/MRI/655_N4_o.nii.gz
2023-11-14 10:52:10,235 INFO image_writer.py:197 - writing: datasets/GIN/MRI/656_N4_o.nii.gz
2023-11-14 10:52:13,433 INFO image_writer.py:197 - writing: datasets/GIN/MRI/660_N4_o.nii.gz
2023-11-14 10:52:16,789 INFO image_writer.py:197 - writing: datasets/GIN/MRI/661_N4_o.nii.gz
2023-11-14 10:52:19,997 INFO image_writer.py:197 - writing: datasets/GIN/MRI/662_N4_o.nii.gz
2023-11-14 10:52:23,306 INFO image_writer.py:197 - writing: datasets/GIN/MRI/663_N4_o.nii.gz
2023-11-14 10:52:26,540 INFO image_writer.py:197 - writing: datasets/GIN/MRI/664_N4_o.nii.gz
2023-11-14 10:52:29,868 INFO image_writer.py:197 - writing: datasets/GIN/MRI/665_N4_o.nii.gz
2023-11-14 10:52:32,973 INFO image_writer.py:197 - writing: datasets/GIN/MRI/667_N4_o.nii.gz
2023-11-14 10:52:36,251 INFO image_writer.py:197 - writing: datasets/G

# Resample in MONAI

In [8]:
target_affine = torch.tensor([
        [ 0.1200,  0.0000,  0.0000, -7.6800],
        [ 0.0000,  0.1200,  0.0000, -7.6800],
        [ 0.0000,  0.0000,  0.1200, -7.6800],
        [ 0.0000,  0.0000,  0.0000,  1.0000]
],
    dtype=torch.float64)
for dataset in datasets:
    print(dataset, end='\r')
    if dataset == 'GIN':
        mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*_o.nii.gz")))
        masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*_o.nii.gz")))
        labels = sorted(glob(os.path.join('datasets', dataset, 'Labels', "*_o.nii.gz")))
        saver_label = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Labels/', output_postfix='128', separate_folder=False)  
        ensure_label = monai.transforms.AsDiscrete(rounding="torchrounding")
    else:
        mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*_N4.nii.gz")))
        masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*_N4.nii.gz")))        
    transform_mri = monai.transforms.SpatialResample(mode="bilinear")
    loader = monai.transforms.LoadImage(ensure_channel_first=True)
    saver_mri = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/MRI/', output_postfix='128', separate_folder=False) 
    saver_mask = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Mask/', output_postfix='128', separate_folder=False)  
    transform_mri = monai.transforms.SpatialResample(mode="bilinear")
    transform_mask = monai.transforms.SpatialResample(mode="nearest")
    ensure_mask = monai.transforms.AsDiscrete(threshold=0.5)
    for i, mri in enumerate(mris):    
        tensor = loader(mri)
        mask = loader(masks[i])
        out_mri = transform_mri(tensor, target_affine, spatial_size=(128,128,128))
        out_mask = transform_mask(mask, target_affine, spatial_size=(128,128,128))
        out_mask = ensure_mask(out_mask)
        saver_mri(out_mri)
        saver_mask(out_mask)
        if dataset == 'GIN':
            label = loader(labels[i])
            out_label = transform_mask(label, target_affine, spatial_size=(128,128,128))
            out_label = ensure_label(out_label)
            saver_label(out_label)

2023-11-14 10:56:17,528 INFO image_writer.py:197 - writing: datasets/GIN/MRI/653_N4_o_128.nii.gz
2023-11-14 10:56:17,863 INFO image_writer.py:197 - writing: datasets/GIN/Mask/653_N4_o_128.nii.gz
2023-11-14 10:56:18,311 INFO image_writer.py:197 - writing: datasets/GIN/Labels/653_N4_o_128.nii.gz
2023-11-14 10:56:19,338 INFO image_writer.py:197 - writing: datasets/GIN/MRI/655_N4_o_128.nii.gz
2023-11-14 10:56:19,692 INFO image_writer.py:197 - writing: datasets/GIN/Mask/655_N4_o_128.nii.gz
2023-11-14 10:56:20,211 INFO image_writer.py:197 - writing: datasets/GIN/Labels/655_N4_o_128.nii.gz
2023-11-14 10:56:21,499 INFO image_writer.py:197 - writing: datasets/GIN/MRI/656_N4_o_128.nii.gz
2023-11-14 10:56:21,855 INFO image_writer.py:197 - writing: datasets/GIN/Mask/656_N4_o_128.nii.gz
2023-11-14 10:56:22,395 INFO image_writer.py:197 - writing: datasets/GIN/Labels/656_N4_o_128.nii.gz
2023-11-14 10:56:23,668 INFO image_writer.py:197 - writing: datasets/GIN/MRI/660_N4_o_128.nii.gz
2023-11-14 10:56:2

# Orientation with MONAI

In [9]:
for dataset in datasets:
    print(dataset, end='\r')
    mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*_128.nii.gz")))
    masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*_128.nii.gz")))
    loader = monai.transforms.LoadImage(ensure_channel_first=True)
    saver_mri = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/MRI/', output_postfix='ras', separate_folder=False)
    saver_mask = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Mask/', output_postfix='ras', separate_folder=False)      
    transform = monai.transforms.Orientationd(keys=["image", "mask"], axcodes='RAS')
    ensure_mask = monai.transforms.AsDiscrete(threshold=0.5)
    
    if dataset == 'GIN':
        labels = sorted(glob(os.path.join('datasets', dataset, 'Labels', "*_128.nii.gz")))
        saver_label = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Labels/', output_postfix='ras', separate_folder=False)  
        ensure_label = monai.transforms.AsDiscrete(rounding="torchrounding")
        
    for i, mri in enumerate(mris):    
        tensor = loader(mri)
        mask = loader(masks[i])
        if dataset == 'GIN':
            label = loader(labels[i])
            dict_tensor = {"image": tensor, "mask": mask, "label": label}
        else:
            dict_tensor = {"image": tensor, "mask": mask}
            
        out = transform(dict_tensor)   
        saver_mri(out["image"])
        out["mask"] = ensure_mask(out["mask"])
        saver_mask(out["mask"])
        if dataset == 'GIN':
            out["label"] = ensure_label(out["label"])
            saver_label(out["label"])

2023-11-14 10:57:48,489 INFO image_writer.py:197 - writing: datasets/GIN/MRI/653_N4_o_128_ras.nii.gz
2023-11-14 10:57:48,809 INFO image_writer.py:197 - writing: datasets/GIN/Mask/653_N4_o_128_ras.nii.gz
2023-11-14 10:57:48,911 INFO image_writer.py:197 - writing: datasets/GIN/Labels/653_N4_o_128_ras.nii.gz
2023-11-14 10:57:49,142 INFO image_writer.py:197 - writing: datasets/GIN/MRI/655_N4_o_128_ras.nii.gz
2023-11-14 10:57:49,517 INFO image_writer.py:197 - writing: datasets/GIN/Mask/655_N4_o_128_ras.nii.gz
2023-11-14 10:57:49,615 INFO image_writer.py:197 - writing: datasets/GIN/Labels/655_N4_o_128_ras.nii.gz
2023-11-14 10:57:49,850 INFO image_writer.py:197 - writing: datasets/GIN/MRI/656_N4_o_128_ras.nii.gz
2023-11-14 10:57:50,231 INFO image_writer.py:197 - writing: datasets/GIN/Mask/656_N4_o_128_ras.nii.gz
2023-11-14 10:57:50,343 INFO image_writer.py:197 - writing: datasets/GIN/Labels/656_N4_o_128_ras.nii.gz
2023-11-14 10:57:50,601 INFO image_writer.py:197 - writing: datasets/GIN/MRI/66

2023-11-14 10:58:05,945 INFO image_writer.py:197 - writing: datasets/GIN/Labels/M8876_N4_o_128_ras.nii.gz


# Truncate + Normalize Image Intensity with MONAI

In [10]:
for dataset in datasets:
    print(dataset, end='\r')
    mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*_ras.nii.gz")))
    masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*_ras.nii.gz")))
    loader = monai.transforms.LoadImage(ensure_channel_first=True)
    saver_mri = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/MRI/', output_postfix='norm', separate_folder=False)  
    transform = monai.transforms.ScaleIntensity(minv=0.0, maxv=1.0)  
    if dataset == 'GIN':
        labels = sorted(glob(os.path.join('datasets', dataset, 'Labels', "*_ras.nii.gz")))
        for label in labels:
            out = label.replace(".nii", "_norm.nii")
            shutil.copyfile(label, out)
    for i, mri in enumerate(mris):            
        tensor = loader(mri)
        out = transform(tensor)  
        out.array[out.array > 0.99] = 0.99
        out.array[out.array < 0.01] = 0.01       
        out = transform(out)  
        saver_mri(out)        
    for mask in masks:
        out = mask.replace(".nii", "_norm.nii")
        shutil.copyfile(mask, out)

2023-11-14 10:58:06,177 INFO image_writer.py:197 - writing: datasets/GIN/MRI/653_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:06,579 INFO image_writer.py:197 - writing: datasets/GIN/MRI/655_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:07,027 INFO image_writer.py:197 - writing: datasets/GIN/MRI/656_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:07,471 INFO image_writer.py:197 - writing: datasets/GIN/MRI/660_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:07,883 INFO image_writer.py:197 - writing: datasets/GIN/MRI/661_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:08,327 INFO image_writer.py:197 - writing: datasets/GIN/MRI/662_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:08,720 INFO image_writer.py:197 - writing: datasets/GIN/MRI/663_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:09,210 INFO image_writer.py:197 - writing: datasets/GIN/MRI/664_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:09,649 INFO image_writer.py:197 - writing: datasets/GIN/MRI/665_N4_o_128_ras_norm.nii.gz
2023-11-14 10:58:10,175 INFO image_writer.py:1

# Change Affine to Identity for further Landmarks with MONAI

In [11]:
identity_affine = np.diag((1, 1, 1, 0))
for dataset in datasets:
    print(dataset, end='\r')
    mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*_norm.nii.gz")))
    masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*_norm.nii.gz")))
    loader = monai.transforms.LoadImage(ensure_channel_first=True)    
    saver_mri = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/MRI/', output_postfix='id', separate_folder=False)  
    saver_mask = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Mask/', output_postfix='id', separate_folder=False) 
    if dataset == 'GIN':
        labels = sorted(glob(os.path.join('datasets', dataset, 'Labels', "*_norm.nii.gz")))
        saver_label = monai.transforms.SaveImage(output_dir='datasets/' + str(dataset) + '/Labels/', output_postfix='id', separate_folder=False)  
        for label in labels:
            tensor = loader(label)
            tensor.affine = identity_affine
            saver_label(tensor)               
    for mri in mris:
        tensor = loader(mri)
        tensor.affine = identity_affine
        saver_mri(tensor)        
    for mask in masks:
        tensor = loader(mask)
        tensor.affine = identity_affine
        saver_mask(tensor)        

2023-11-14 10:58:17,532 INFO image_writer.py:197 - writing: datasets/GIN/Labels/653_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:17,656 INFO image_writer.py:197 - writing: datasets/GIN/Labels/655_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:17,767 INFO image_writer.py:197 - writing: datasets/GIN/Labels/656_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:17,883 INFO image_writer.py:197 - writing: datasets/GIN/Labels/660_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:17,995 INFO image_writer.py:197 - writing: datasets/GIN/Labels/661_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:18,111 INFO image_writer.py:197 - writing: datasets/GIN/Labels/662_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:18,223 INFO image_writer.py:197 - writing: datasets/GIN/Labels/663_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:18,335 INFO image_writer.py:197 - writing: datasets/GIN/Labels/664_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:18,444 INFO image_writer.py:197 - writing: datasets/GIN/Labels/665_N4_o_128_ras_norm_id

2023-11-14 10:58:34,327 INFO image_writer.py:197 - writing: datasets/GIN/Mask/M867_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:34,452 INFO image_writer.py:197 - writing: datasets/GIN/Mask/M872_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:34,567 INFO image_writer.py:197 - writing: datasets/GIN/Mask/M874_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:34,706 INFO image_writer.py:197 - writing: datasets/GIN/Mask/M875_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:34,827 INFO image_writer.py:197 - writing: datasets/GIN/Mask/M877_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:34,943 INFO image_writer.py:197 - writing: datasets/GIN/Mask/M886_N4_o_128_ras_norm_id.nii.gz
2023-11-14 10:58:35,063 INFO image_writer.py:197 - writing: datasets/GIN/Mask/M8876_N4_o_128_ras_norm_id.nii.gz


# Deformable registration datasets to P56 Allen Atlas with ANTS

In [ ]:
for dataset in datasets:
    print(dataset, end='\r')
    mris = sorted(glob(os.path.join('datasets', dataset, 'MRI', "*_id.nii.gz"))) 
    masks = sorted(glob(os.path.join('datasets', dataset, 'Mask', "*_id.nii.gz")))    
    labels = sorted(glob(os.path.join('datasets', dataset, 'Labels', "*_id.nii.gz")))   
    atlas = os.path.join('Atlas', 'P56_Atlas_128_norm_id.nii.gz')
    for i, mri in enumerate(mris): 
        if not os.path.exists(masks[i].replace("_id.nii", "_id_affine_deformable.nii")):
            print(mri, end='\r')
            command = ['antsRegistration']
            command.extend(['-d', '3'])
            command.extend(['-o', '[<Prefix>,<Output>,<OutputInverse>]'])
            command.extend(['-u', '1'])
            command.extend(['-r', '[<Fixed>,<Moving>,1]'])

            command.extend(['-t', 'Rigid[0.1]'])
            command.extend(['-m', 'MI[<Fixed>,<Moving>,1,32,Regular,0.2]'])
            command.extend(['-c', '[2000x2000x2000,1e-9,15]'])
            command.extend(['-s', '2x1x0'])    
            command.extend(['-f', '4x2x1']) 

            command.extend(['-t', 'Affine[0.1]'])
            command.extend(['-m', 'MI[<Fixed>,<Moving>,1,32,Regular,0.1]'])
            command.extend(['-c', '[2000x2000x2000,1e-9,15]'])
            command.extend(['-s', '2x1x0'])    
            command.extend(['-f', '4x2x1']) 

            command.extend(['-t', 'SyN[0.15,3.0,0.0]'])
            command.extend(['-m', 'CC[<Fixed>,<Moving>,1,4]'])
            command.extend(['-c', '[100x100x100x50,1e-9,15]'])
            command.extend(['-s', '3x2x1x0'])    
            command.extend(['-f', '6x4x2x1']) 

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", mri)
                cmd = cmd.replace("<Prefix>", "deformable_")
                cmd = cmd.replace("<Output>", mri.replace("_id.nii", "_id_affine_deformable.nii"))
                cmd = cmd.replace("<OutputInverse>", mri.replace("_id.nii", "_id_affine_deformable_inverse.nii"))
                command[j] = cmd

            # Get .mat (affine), warp (deformable) and warped moving image (to atlas space)
            subprocess.call(command)     

            # Move affine .mat and rename to correct name
            shutil.move("deformable_0GenericAffine.mat", mri.replace("_id.nii.gz", "_id_affine.mat"))         
            # Move warp .nii.gz and rename to correct name
            shutil.move("deformable_1Warp.nii.gz", mri.replace("_id.nii.gz", "_id_affine_warp.nii.gz")) 
            shutil.move("deformable_1InverseWarp.nii.gz", mri.replace("_id.nii.gz", "_id_affine_warp_inverse.nii.gz")) 

            # Apply .mat to MRI to get AFFINE MRI
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', '<Moving>'])
            command.extend(['-r', '<Fixed>'])
            command.extend(['-o', '<Output>'])
            command.extend(['-n', '<Interpolation>'])
            command.extend(['-t', '<AffineTransform>'])

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", mri)
                cmd = cmd.replace("<Output>", mri.replace("_id.nii", "_id_affine.nii"))            
                cmd = cmd.replace("<Interpolation>", "Linear")
                cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_affine.mat"))  
                command[j] = cmd

            subprocess.call(command) 

            # Apply .mat to Mask to get AFFINE MASK
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', '<Moving>'])
            command.extend(['-r', '<Fixed>'])
            command.extend(['-o', '<Output>'])
            command.extend(['-n', '<Interpolation>'])
            command.extend(['-t', '<AffineTransform>'])

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", masks[i])
                cmd = cmd.replace("<Output>", masks[i].replace("_id.nii", "_id_affine.nii"))            
                cmd = cmd.replace("<Interpolation>", "NearestNeighbor")
                cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_affine.mat"))  
                command[j] = cmd

            subprocess.call(command) 

            # Apply .mat and warp to Mask to get DEFORMABLE MASK
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', '<Moving>'])
            command.extend(['-r', '<Fixed>'])
            command.extend(['-o', '<Output>'])
            command.extend(['-n', '<Interpolation>'])
            command.extend(['-t', '<DeformableTransform>'])
            command.extend(['-t', '<AffineTransform>'])

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", masks[i])
                cmd = cmd.replace("<Output>", masks[i].replace("_id.nii", "_id_affine_deformable.nii"))            
                cmd = cmd.replace("<Interpolation>", "NearestNeighbor")
                cmd = cmd.replace("<DeformableTransform>", mri.replace("_id.nii.gz", "_id_affine_warp.nii.gz"))  
                cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_affine.mat"))  
                command[j] = cmd

            subprocess.call(command) 


            # Apply .mat to Labels to get AFFINE LABELS
            if dataset == 'GIN':            
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', '<Moving>'])
                command.extend(['-r', '<Fixed>'])
                command.extend(['-o', '<Output>'])
                command.extend(['-n', '<Interpolation>'])
                command.extend(['-t', '<AffineTransform>'])

                for j, cmd in enumerate(command):
                    cmd = cmd.replace("<Fixed>", atlas)
                    cmd = cmd.replace("<Moving>", labels[i])
                    cmd = cmd.replace("<Output>", labels[i].replace("_id.nii", "_id_affine.nii"))            
                    cmd = cmd.replace("<Interpolation>", "MultiLabel")
                    cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_affine.mat"))  
                    command[j] = cmd

                subprocess.call(command) 

                # Apply .mat and warp to Labels to get DEFORMABLE LABELS                          
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', '<Moving>'])
                command.extend(['-r', '<Fixed>'])
                command.extend(['-o', '<Output>'])
                command.extend(['-n', '<Interpolation>'])
                command.extend(['-t', '<DeformableTransform>'])
                command.extend(['-t', '<AffineTransform>'])

                for j, cmd in enumerate(command):
                    cmd = cmd.replace("<Fixed>", atlas)
                    cmd = cmd.replace("<Moving>", labels[i])
                    cmd = cmd.replace("<Output>", labels[i].replace("_id.nii", "_id_affine_deformable.nii"))            
                    cmd = cmd.replace("<Interpolation>", "MultiLabel")
                    cmd = cmd.replace("<DeformableTransform>", mri.replace("_id.nii.gz", "_id_affine_warp.nii.gz"))  
                    cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_affine.mat"))  
                    command[j] = cmd

                subprocess.call(command) 